In [103]:
import pandas as pd
import numpy as np

In [104]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

/home/muxail/.cache/pypoetry/virtualenvs/raif-hack-rVtTtMQz-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [105]:
df = pd.concat([train, test])

In [106]:
def get_floor(x):
    try:
        x = float(x)
        if np.isnan(x):
            return 'NAN'
        else:
            x = str(int(x))
            return x
    except ValueError:
        if ',' in x:
            splitted_row = x.split(',')
            new_row = []
            for word in splitted_row:
                if ' ' in word:
                    new_row.append(word.replace(' ', ''))
                new_row.append(word)
            x = ' '.join(new_row)
        x = x.replace('  ', ' ').lower()
        dict_replace = {
            'подвал 1 1': '-1 1',
            'подвал': '-1',
            'цоколь 1 1': '-2 1',
            '1 2 антресоль': '1 2 +1',
            'цоколь': '-2',
            'тех.этаж (6)': '+2',
            'фактически на уровне 1 этажа': '1',
            '1 2 3': '1 2 3', 
            '1 подвал подвал': '-1 1',
            '1 2 3 4': '1 2 3 4', 
            '1 2': '1 2',
            '1 2 3 4 5': '1 2 3 4 5', 
            '5 мансарда мансарда': '5 +3',
            '1-й подвал подвал': '-1 1', 
            '1 подвал подвал антресоль антресоль': '-1 1 +1',
            'мезонин': 'NAN', 
            'подвал 1-3 1-3': '-1 1 2 3', 
            '1 (цокольный этаж)': '-2 1',
            '3 мансарда(4эт) мансарда (4 эт)': '3 +3',
            'подвал 1': '-1 1',
            '1 антресоль антресоль': '1 +1', 
            '1-3': '1 2 3',
            'мансарда (4эт)': '4 +3', 
            '1 2. 2.': '1 2',
            'подвал подвал 1 1 ': '-1 1', 
            '1 2 2': '1 2', 
            'подвал 1 1 2 3': '-1 1 2 3',
            '1 + подвал (без отделки)': '-1 1', 
            'мансарда': '+3', 
            '2 3': '2 3', 
            '4 5 5': '4 5',
            '1-й 2-й 2-й': '1 2',
            '1этаж 1 этаж подвал подвал': '-1 1',
            '1 цоколь цоколь': '-2 1',
            'подвал 1-7 1-7 техэтаж техэтаж': '-1 1 2 3 4 5 6 7 +4', 
            '3 (антресоль)': '3 +1', 
            '1 2 2 3 3': '1 2 3',
            'цоколь 1 1 2(мансарда)': '-2 1 2 +3',
            'подвал 3.4этаж 3. 4 этаж': '-1 3 4',
            'подвал 1-4этаж 1-4 этаж': '-1 1 2 3 4', 
            'подва 1.2этаж 1.2 этаж': '-1 1 2', 
            '2 3 3': '2 3',
            '7 8': '7 8', 
            '1 этаж': '1',
            '1-й': '1', 
            '3 этаж': '3',
            '4 этаж': '4', 
            '5 этаж': '5',
            'подвал 1 2 3 4 5': '-1 1 2 3 4 5', 
            'подвал цоколь цоколь 1этаж 1 этаж': '-2 1',
            '3 мансарда мансарда': '3 +3', 
            'цоколь 1 1 2 2 3 4 5 6': '-2 1 2 3 4 5 6',
            '1 1 2 2 антресоль антресоль': '1 2 +1',
            '3этаж 3 этаж мансарда(4этаж) мансарда (4 этаж)': '3 4 +3',
            'цокольный': '-2',
            '1 2 2 ': '1 2', 
            '3 4': '3 4', 
            'подвал 1и4этаж 1 и 4 этаж': '-1 1 4', 
            '5(мансарда)': '5 +3',
            'техническийэтаж технический этаж 5 6': '+4 5 6',
            '1-2 1-2 подвальный подвальный': '-1 1 2', 
            '1 2 2 3 3 мансардный мансардный': '1 2 3 +3',
            'подвал 1 1 2 2 3 3': '-1 1 2 3',
            '1 2 3 антресоль антресоль техническийэтаж технический этаж': '1 2 3 +1 +4',
            '3 4 4': '3 4',
            '1-3этажи 1-3 этажи цоколь(188 цоколь (188 4кв.м) 4 кв.м) подвал(104кв.м) подвал (104 кв.м)': '-2 -1 1 2 3',
            '1 2 3 4 подвал подвал': '-1 1 2 3', 
            '2-й': '2',
            '1 2этаж 2 этаж': '1 2',
            'подвал 1 1 2 2': '-1 1 2',
            '1-7': '1 2 3 4 5 6 7', 
            '1 (по док-м цоколь)': '-2', 
            '1 2 подвал подвал ': '-1 1 2', 
            'подвал 2 2': '-1 2',
            'подвал 1 2 3': '-1 1 2 3', 
            '1 2 3этаж 3 этаж подвал подвал': '-1 1 2 3',
            '2 3 3 4 4 тех.этаж тех.этаж': '2 3 4 +4', 
            'цокольный 1 1 2': '-2 1 2',
            'техническое подполье': '-1'
        }
        return dict_replace.get(x, 'nasnsadnasdasd')

In [107]:
from sklearn.feature_extraction.text import CountVectorizer

In [108]:
countvectorizer = CountVectorizer(min_df=50)
countvectorizer.fit(df['floor'].apply(get_floor).to_list())

CountVectorizer(min_df=50)

In [109]:
floors = pd.DataFrame(
    countvectorizer.transform(df['floor'].apply(get_floor).to_list()).A, 
    columns=[f"floor_{i}" for i in range(len(countvectorizer.vocabulary_))]
)

floors['id'] = df['id'].values

In [110]:
floors.to_csv('../data/floors.csv', index=False)